In [62]:
import pandas as pd
import defusedxml.ElementTree as ET
import re

In [4]:
skoly_data = []
zarizeni_info_data = []
zarizeni_misto_data = []
skola_obor_data = []

for ps in root.findall('.//PravniSubjekt'):
    # TAB "skoly" a "zarizeni_info"
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text

        raw_skoly = {'IZO' : izo,  
                    'ZarizeniTyp': typZarizeni, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka,           
                    }
        raw_zarizeni_info = {
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'jazyk' : jazyk, 
                    'IZO' : izo,
                    }
        skoly_data.append(raw_skoly)
        zarizeni_info_data.append(raw_zarizeni_info)

        # TAB "zarizeni_misto"
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            idMista = smvc.find('IDMista').text
            adresaX = smvc.find('MistoAdresa2').text
            adresaY = smvc.find('MistoAdresa3').text
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace
            # adresu možná nahradit přes (nutno poupravit kod) RUIAN API, nebo https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2F00025712%2F59ddccd404a617f695e3a1ef8e65b81e 
                             
            raw_zarizeni_misto = {
                    'druh' : druh,
                    'IDMista': idMista,
                    'adresaX' : adresaX,
                    'adresaY' : adresaY,
                    'IZO' : izo,
                   }
            zarizeni_misto_data.append(raw_zarizeni_misto)
            
        # TAB "skola_obor"
        if sz.find('SkolaOboryVzdelani') is not None:
            for sov in sz.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
                oborNazev = sov.find('OborNazev').text
                delkaVzdelavani = sov.find('DelkaVzdelavani').text
                oborKapacita = sov.find('OborKapacita').text
                oborDobihajici = sov.find('OborDobihajici').text
                # print(oborNazev)
    
                raw_skola_obor = {
                    'OborNazev' : oborNazev,
                    'DelkaVzdelavani': delkaVzdelavani,
                    'OborKapacita' : oborKapacita,
                    'OborDobihajici' : oborDobihajici,
                    'IZO' : izo,
                   }
                skola_obor_data.append(raw_skola_obor)

### Školy (kapacita) data -> dataframe dle ERD

In [5]:
skoly = pd.DataFrame(skoly_data)

zarizeni_info = pd.DataFrame(zarizeni_info_data)
zarizeni_info.insert(loc=0, column='ID', value=range(1, len(zarizeni_info_data) + 1))
    
zarizeni_misto = pd.DataFrame(zarizeni_misto_data)
zarizeni_misto.insert(loc=0, column='ID', value=range(1, len(zarizeni_misto) + 1))
    
skola_obor = pd.DataFrame(skola_obor_data)
skola_obor.insert(loc=0, column='ID', value=range(1, len(skola_obor_data) + 1))

# kontrola dat u "skoly" a "zarizeni_info" v pořádku - stejný počet záznamů

## MŠ + PSZŠSpeciální (kapacita) TAB

In [6]:
materske_skoly = skoly[skoly['ZarizeniTyp'].isin(["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"])]
materske_skoly = materske_skoly.astype({'Kapacita':'int'})
# materske_skoly.info()
# materske_skoly['Kapacita'].sum()
materske_skoly

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
0,049625918,Mateřská škola,54,dítě
2,107500884,Mateřská škola,70,dítě
4,110034384,Mateřská škola,6,dítě
6,060437171,Mateřská škola,130,dítě
8,110035585,Mateřská škola,18,dítě
...,...,...,...,...
27171,181145014,Mateřská škola (lesní mateřská škola),15,dítě
27177,181145499,Mateřská škola (lesní mateřská škola),15,dítě
27181,181145871,Mateřská škola,44,dítě
27187,181146878,Mateřská škola (lesní mateřská škola),16,dítě


## DětskéSkupinky (realtime kapacita) TAB

In [7]:
from web_scrap_DS import DS_tabulka

In [8]:
DS_tabulka = DS_tabulka.astype({'Kapacita':'int'})
# DS_tabulka.info()
DS_tabulka

,ZarizeniTyp,Kapacita,Kraj
0,Dětská skupinka,4455,Hlavní město Praha
1,Dětská skupinka,1305,Jihočeský
2,Dětská skupinka,4029,Jihomoravský
3,Dětská skupinka,256,Karlovarský
4,Dětská skupinka,1137,Královéhradecký
5,Dětská skupinka,758,Liberecký
6,Dětská skupinka,2186,Moravskoslezský
7,Dětská skupinka,1387,Olomoucký
8,Dětská skupinka,1044,Pardubický
9,Dětská skupinka,1053,Plzeňský


## Předškolní vzdělávání a péče TAB (kapacita a realtime kapacita)

In [9]:
predskolniVP = pd.concat([materske_skoly, DS_tabulka], ignore_index=True, sort=False)
predskolniVP.info()
predskolniVP

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5501 entries, 0 to 5500
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5501 non-null   object
 2   Kapacita          5501 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              14 non-null     object
dtypes: int64(1), object(4)
memory usage: 215.0+ KB


,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka,Kraj
0,049625918,Mateřská škola,54,dítě,NaN
1,107500884,Mateřská škola,70,dítě,NaN
2,110034384,Mateřská škola,6,dítě,NaN
3,060437171,Mateřská škola,130,dítě,NaN
4,110035585,Mateřská škola,18,dítě,NaN
...,...,...,...,...,...
5496,NaN,Dětská skupinka,1053,NaN,Plzeňský
5497,NaN,Dětská skupinka,3986,NaN,Středočeský
5498,NaN,Dětská skupinka,1551,NaN,Ústecký
5499,NaN,Dětská skupinka,591,NaN,Vysočina


In [10]:
predskolniVP.to_csv('predskolniVP.csv', mode='w')

## Porodnost 2018-2023 TAB (neměnná data z webu ČSÚ)

In [11]:
from web_scrap_porodnost import porodnost_tabulka

In [12]:
porodnost_tab = porodnost_tabulka.astype({"PocetNarozenychDeti":'int'}) 
porodnost_tabCR = porodnost_tab[porodnost_tab['Kraj'] == 'Česká republika']
porodnost_tab_kraje = porodnost_tab[porodnost_tab['Kraj'] != 'Česká republika']
# porodnost_tab_kraje
# porodnost_tabCR[porodnost_tabCR['Rok'].isin([2021, 2020, 2019, 2018])]['PocetNarozenychDeti'].sum()
# porodnost_tab_kraje.info()

In [13]:
porodnost_tab_kraje.to_csv('porodnost.csv', mode='w')

## ČSÚ: Školy a školská zařízení - reálný počet žáků

### Pro rozbor ZŠ, Do 1třídy nastupuje více dětí, než kolik mělo odklad. Vzhledem k povinnému poslednímu ročníku v MŠ se pravděpodobně jedná o imigraci?

In [21]:
# csu = pd.read_excel(r'D:\git\skolstvi_projekt\CSU\230042230202.xlsx', sheet_name=['2.2.17', '2.2.18'], header = 3, index_col=0, usecols='A, H:L')
# # 2.2.17 -> počet žáků nově přijatých do 1. ročníku celkem
# # 2.2.18 -> počet žáků 7letých a starších nově přijatých do 1. ročníku, podmnožina 2.2.17
# # pro pozdější užití, na kartě 2.2.20 -> žáci opakující ročník, údaj "z toho v 1.ročníku" nutno PŘIČÍST k údaji z 2.2.17, 
# #žel tento údaj nění formátovaný v časocé řadě, nutno pro každý ročník explicitně stáhnout samostatný xlsx soubor

# nPrijatiZS = pd.DataFrame(csu['2.2.17'])
# nPrijatiZS_7plus = pd.DataFrame(csu['2.2.18'])
# # nPrijatiZS
# # nPrijatiZS_7plus

### Pro rozbor MŠ

In [155]:
def get_MS_dochazka(path, sheet='1.1.9'):
    dochazka = (pd.read_excel(MS_path, sheet_name=sheet, header=3, index_col=0, usecols='A, C, E, G, I, K')).iloc[3:-1]
    
    rok_match = re.search(r'(\d{4}-\d{2})', path)
    if rok_match:
        year = rok_match.group(1).replace('-', '/')  # Extrahovaný a upravený školní rok
    else:
        year = "Rok nebyl nalezen"
    
    dochazka.insert(loc=5, column='rok', value=year)
    return dochazka

MS_23_24 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2023-24.xlsx')
MS_22_23 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2022-23.xlsx')
MS_21_22 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2021-22.xlsx')
MS_20_21 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2020-21.xlsx')
MS_19_20 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2019-20.xlsx')
# MS_18_19 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2018-19_konkrTab.xlsx', sheet='2300421909') #prozatím nefunguje, problém s samotném souboru? 

MS_18_19 = (pd.read_excel(r'D:\git\skolstvi_projekt\CSU\MS2018-19_konkrTab.xlsx', sheet_name='2300421909', header=3, index_col=0, usecols='A, C, E, G, I, K')).iloc[3:-1]
MS_18_19.insert(loc=5, column='rok', value='2018/19')

# MS_all = pd.concat([MS_23_24, MS_22_23, MS_21_22], ignore_index=False, sort=False)
# MS_all